In [1]:
import pandas as pd
import numpy as np
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve 

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.svm import SVC

from sklearn.model_selection import KFold, cross_val_score

import copy
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Setup Dataframe

In [2]:
df = pd.read_csv("Covid-19_dataset.csv",header=1)
df['is_Male'] = pd.get_dummies(df['Gender'])['MALE']#one hot encoding gender so male = #1 and female = 0
df['is_Female'] = pd.get_dummies(df['Gender'])['FEMALE']

In [3]:
df_bloodType = pd.get_dummies(df['Blood_Group'])

# Remove unwanted features and set up One Hot Encoding

In [4]:
blacklist = ['Age',' Incubation period (days)','Blood_Group','CONTROL_blood_group','Risk_Factor','Infection_type','Gender','Place_of_isolation','Patient_In','Patient_out']
labels = ['Severeity','Outcome']
df = df.drop(blacklist,axis=1)
df

,Severeity,Outcome,Fever,Chills,Cough,Dyspnea,Anosmia_Ageusia,Loss_of_appetite,Asthenia,Headache,Muscle_ache,Cyanosis,Rhinorrhea,Sore_throat,Diarrhea,Nausea_vomiting,is_Male,is_Female
0,Moderate,Recovered,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0
1,Mild,Recovered,1,0,1,0,1,1,1,1,1,0,0,1,1,1,1,0
2,Severe,Died,1,1,1,1,0,0,1,1,0,1,0,1,0,0,1,0
3,Moderate,Recovered,1,0,1,0,1,1,1,1,1,0,1,0,1,0,1,0
4,Moderate,Recovered,0,0,1,0,1,1,1,1,1,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,Moderate,Recovered,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,0
5664,Severe,Died,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
5665,Severe,Died,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0
5666,Severe,Died,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0


In [5]:
df = pd.concat([df,df_bloodType],axis=1)

In [6]:
all_columns = set(df.columns)
all_columns = all_columns - set(['Severeity','Outcome'])
all_columns = list(all_columns)

# Reset cells to remove empty values

In [7]:
# getting rid of blank values
print("before",df.shape)
for name in all_columns:
    df = df[df[name] != " "]
    df = df.dropna()
    
label_Severeity = df['Severeity'] #Severeity vs Outcome
label_Outcome = df['Outcome']
df = df.drop(['Severeity','Outcome'],axis=1)


df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))#strings to numeric
all_columns = df.columns
# df = df.reset_index()

print("after",df.shape)


before (5668, 26)
after (5657, 24)


# Boruta

In [16]:
model = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5,random_state=1)

feat_selector = BorutaPy(
        verbose=2,
        estimator=model,
        n_estimators='auto',
        alpha=0.1)

In [17]:
feat_selector.fit(np.array(df),label_Outcome)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	24
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	6
Rejected: 	18
Iteration: 	7 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	8 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	9 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	10 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	11 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	12 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	13 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	14 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	15 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
Iteration: 	16 / 100
Confirmed: 	1
Tentative: 	5
Rejected: 	18
I

BorutaPy(alpha=0.1,
         estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=63, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x1E10CC4DA40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1E10CC4DA40, verbose=2)

In [18]:
feat_selector.support_

array([False,  True, False, False, False, False, False, False, False,
        True, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False])

In [19]:
feat_selector.support_weak_

array([False, False,  True, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [20]:
support = []
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        print("Passes the test: ", df.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
        support.append(df.columns[i])


for i in range(len(feat_selector.support_weak_)):
    if feat_selector.support_weak_[i]:
        print("May pass the test: ", df.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
        support.append(df.columns[i])

Passes the test:  Chills  - Ranking:  1
Passes the test:  Cyanosis  - Ranking:  1
Passes the test:  Diarrhea  - Ranking:  1
May pass the test:  Cough  - Ranking:  2
May pass the test:  Sore_throat  - Ranking:  2


In [21]:
support

['Chills', 'Cyanosis', 'Diarrhea', 'Cough', 'Sore_throat']